[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

# Introducción a *Dataframes*.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Intro a Dataframes").getOrCreate()
ct = spark.sparkContext

Los *dataframes* son un concepto compartido entre plataformas como *R*, *Pandas* y *Scala*. Son estructuras tabulares en las que todos los datos de una columna comparten el mismo tipo de datos. Cada columna tiene un título y cada renglón tiene un índice. A la descripción de los tipos de datos de cada columna de un *dataframe* se le conoce como esquema (*schema*).

*PySPark* tiene la capacidad de poder manejar *dataframes* tanto de *Pandas* como de *Spark* e incluso cuenta con una [*API*](https://spark.apache.org/docs/latest/api/python/user_guide/pandas_on_spark/index.html) que optimiza la interacción entre ambos tipos de *dataframes*.

## Los *dataframes* de *Spark*.

Un *dataframe* de *Spark* es un objeto instanciado de la clase [```pyspark.sql.DataFrame```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html).

### Creación de *dataframes*.

La función [```spark.createDataFrame()```](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html) permite crear dataframes a partir de objetos que se ingresan como argumentos.


```pyspark
df = spark.createDataFrame(data=<obj>, <títulos columnas>, schema=<esquema>)
```

Donde:

* <obj> es un objeto que represente una estructura tabular el cual puede ser:
    * Una colección de *Python*.
    * Un *dataframe* de *Pandas*.
    * Un *RDD* de *Spark*.
    
Cabe hacer notar que los *dataframes* de *Spark* se construyen de forma perezosa, por lo que aún cuando sean definidos, estos no serán creados hasta que sean requeridos.

### El método ```df.show()```.

El método ```df.show()``` muestra los primeros ```n``` números de un *dataframe* de *Spark*.

```
df.show(<n>)
```

Donde:

* ```<n>``` es el número de renglones desplegados. El valor por defecto es ```20```.

**Ejemplo:**

In [ ]:
import pandas as pd

In [ ]:
pandas_df = pd.DataFrame({'Dirección':('Sur', 'Norte', 'Sur', 'Este'),
              'Rumbo':('Este', 'Noroeste', 'Norte', 'Norte'),
             'Pasajeros':(12, 24, 32, 5),
             'Documentado':(True, None, True, False) })

In [ ]:
df = spark.createDataFrame(pandas_df)

In [ ]:
df.show()

In [ ]:
df.show(2)

In [ ]:
from pyspark.sql import Row

In [ ]:
rdd = ct.parallelize((Row('Sur', 'Este', '12', True),
                     Row('Norte', 'Noroeste', '24', None),
                     Row('Sur', 'Norte', '32', True),
                     Row('Este', 'Norte', '5', False)))

In [ ]:
df = spark.createDataFrame(rdd, ['Dirección', 'Rumbo', 'Pasajeros', 'Documentado'])

In [ ]:
df.show()

### Selección de una columna de un *dataframe*.

```
df.<Nombre Columna>
```

```
df[<n>]
```


In [ ]:
df.Rumbo

In [ ]:
columna = df[1]

### El atributo ```df.schema```.

In [ ]:
df.schema

### El método ```df.printSchema()```

In [ ]:
df.printSchema()

## Tipado de *Spark*.

*Spark* cuenta con distintos tipos de datos que extienden a los tipo nativos de *Scala*, *Python* y *R*. El módulo ```pyspark.sql.types``` contiene a todas las clases correspondientes a dichos tipo.


https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [ ]:
from pyspark.sql.types import *

## Definición de *schemas* .

La clase ```StructField```.

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructField.html

La clase ```StructType```.

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.types.StructType.html

In [ ]:
schema = StructType([StructField('Dirección', StringType(), True), 
                     StructField('Rumbo', StringType(), True), 
                     StructField('Pasajeros', ByteType(), True), 
                     StructField('Documentado', BooleanType(), True)])

In [ ]:
df = spark.createDataFrame(data=pandas_df, schema=schema)
df

In [ ]:
df.show()

## Lectura de archivos para *dataframes*.

[```spark.read```](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.SparkSession.read.html)

### Fuentes de datos compatibles para los *dataframes*.

https://spark.apache.org/docs/latest/sql-data-sources.html

In [ ]:
df = spark.read.parquet('data/data_covid.parquet')

In [ ]:
df

In [ ]:
df.show()

In [ ]:
df.show(1)

In [ ]:
df.toPandas()

## Ejemplo de funciones de SQL.

In [ ]:
df.select('index','AGUASCALIENTES').where(df.AGUASCALIENTES >1000).show()

In [ ]:
from datetime import datetime

In [ ]:
df.select('index','AGUASCALIENTES').where(df.index == datetime(2020,12,20)).show()

In [ ]:
spark.stop()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2022.</p>